In [100]:
from bs4 import Tag, NavigableString, BeautifulSoup
from pprint import pprint
import urllib

In [253]:
# html_doc = urllib.request.urlopen("https://en.wikipedia.org/wiki/Guns_N%27_Roses").read()
# html_doc = urllib.request.urlopen("https://en.wikipedia.org/wiki/The_Beatles").read()
# html_doc = urllib.request.urlopen("https://en.wikipedia.org/wiki/The_Action_Design").read()
html_doc = urllib.request.urlopen("https://en.wikipedia.org/wiki/Babes_in_Toyland_(band)").read()

soup = BeautifulSoup(html_doc, 'html.parser')

In [254]:
def get_info_from_cell(cell):
    
    # if the cell contains a <ul> tag then extracting the info as an array is simple
    if cell.find('ul'):
        return list(map(lambda e: e.text, cell.find('ul').find_all('li')))
    
    # if the cell does not contain a <ul> tag, then do the following
    
    # split the text in the cell by <br/> tags
    cell_info = ['']
    for element in cell.contents:
        if isinstance(element, NavigableString):
            cell_info[-1] += str(element)
        elif element.name == 'br':
            cell_info.append('')
        else:
            cell_info[-1] += str(element.text)        
    
    # further split the text by newline characters
    cell_info_2 = []
    for string in cell_info:
        cell_info_2 += string.split('\n')
    
    # remove any empty strings
    cell_info_3 = [s for s in cell_info_2 if s != '']
    
    return cell_info_3

In [255]:
def parse_infobox(soup):
    
    infobox_dict = {}

    infobox_rows = soup.find("table", {"class": "infobox"}).tbody.contents
    for row in infobox_rows:
        if len(row.contents) != 2:
            continue
        cell_1 = row.contents[0]
        cell_2 = row.contents[1]
        if cell_1.name != "th" or cell_2.name != "td":
            continue

        infobox_dict[cell_1.text] = get_info_from_cell(cell_2)
    
    return infobox_dict

In [256]:
pprint(parse_infobox(soup))

{'Associated acts': ['Sugar Babydoll',
                     'Pagan Babies',
                     'Italian Whorenuns',
                     'Crunt',
                     'Koalas',
                     'Katastrophy Wife'],
 'Genres': ['Alternative rock[1]', 'punk rock[2]', 'post-punk[3]'],
 'Labels': ['Twin Tone', 'Southern', 'Strange Fruit', 'Reprise', 'Insipid'],
 'Members': ['Kat Bjelland', 'Lori Barbero', ' Clara Salyer'],
 'Origin': ['Minneapolis, Minnesota, U.S.'],
 'Past members': ['Maureen Herman',
                  ' Michelle Leon',
                  'Chris Holetz',
                  'Cindy Russell',
                  'Dana Cochrane',
                  'Jessie Farmer'],
 'Years active': ['1987–2001; 2014–2020']}
